In [ ]:
%load_ext autoreload
%autoreload 2

import hubmapbags
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime
import seaborn as sns
import numpy as np

In [ ]:
token = 'TOKEN'
hubmapbags.utilities.clean()
hubmapbags.reports.daily(token=token)

In [ ]:
#get daily report
report_output_directory = "daily-report"
if not Path(report_output_directory).exists():
    Path(report_output_directory).mkdir()

now = datetime.now()
report_output_filename = f'{report_output_directory}/{str(now.strftime("%Y%m%d"))}.tsv'
print(report_output_filename)

df = pd.read_csv(report_output_filename, sep="\t")

In [ ]:
#dataframe keys
df.keys()

# Plots by Group Name

In [ ]:
df['group_name'].unique()

In [ ]:
group = df.groupby(['group_name', 'status']).count()[['data_type']]
print(group.to_markdown())

In [ ]:
hubmapbags.plots.by_group(df)

# Plots by Data Type

In [ ]:
df['data_type'].unique()

In [ ]:
data_type = df.groupby(['data_type', 'status']).count()
data_type

In [ ]:
df['status'].unique()

In [ ]:
hubmapbags.plots.by_data_type(df)

# Plots by Date

In [ ]:
hubmapbags.plots.by_date(df)